In [1]:
from google.cloud import bigquery as bq
import os
import pandas as pd
import pandas_gbq as pgbq
import util_functions as util
import matplotlib.pyplot as plt

%matplotlib inline

if os.environ['COMPUTERNAME'] == 'DESKTOP-9KG2MPJ':
    service_credentials = 'Service_Credentials/Big Query Horse Play-f8b9319b2129.json'
else:
    service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

ga_data_client = bq.Client(project='bigquery-public-data')

In [2]:
ga_dataset_ref = util.get_dataset(ga_data_client, 'google_analytics_sample')

---------------------------------------
Dataset ID: google_analytics_sample
Friendly Name: None
Full ID: bigquery-public-data:google_analytics_sample
Labels: {}
Project: bigquery-public-data
Ref: DatasetReference('bigquery-public-data', 'google_analytics_sample')
---------------------------------------


In [3]:
# Simple query to ensure I'm getting data out
query = (
"""
SELECT
    #visitId,
    trafficSource.campaign,
    device.deviceCategory,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`
WHERE
    trafficSource.campaign != "(not set)"
GROUP BY
    #visitId,
    trafficSource.campaign,
    device.deviceCategory
""")

In [4]:
df = pgbq.read_gbq(query, dialect='standard')

In [5]:
df

,campaign,deviceCategory,Visits,PVs,NewVisits
0,Data Share Promo,desktop,6,17,4
1,AW - Dynamic Search Ads Whole Site,desktop,22,110,17
2,AW - Dynamic Search Ads Whole Site,tablet,8,52,6
3,Data Share Promo,tablet,1,4,1
4,AW - Dynamic Search Ads Whole Site,mobile,17,105,13
5,Data Share Promo,mobile,5,42,3


In [6]:
# Testing partitioned tables and table suffix
query_mult_days = (
"""
SELECT
    date,
    device.deviceCategory,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX BETWEEN '20170101'
    AND
    '20170105'
GROUP BY
    date,
    device.deviceCategory
ORDER BY
    date ASC,
    Visits DESC
""")

In [7]:
mult_day_df = pgbq.read_gbq(query_mult_days, dialect='standard')

In [8]:
mult_day_df.head(3)

,date,deviceCategory,Visits,PVs,NewVisits
0,20170101,desktop,759,3246,576
1,20170101,mobile,503,1678,390
2,20170101,tablet,102,438,76


In [9]:
# Vs same thing using UNION ALL
query_mult_days_union = (
"""
WITH mult_ga_table AS (
    SELECT
        date,
        device.deviceCategory,
        SUM(totals.visits) AS Visits,
        SUM(totals.pageviews) AS PVs,
        SUM(totals.newVisits) AS NewVisits
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`
    GROUP BY
        date,
        device.deviceCategory
    
    UNION ALL
    
    SELECT
        date,
        device.deviceCategory,
        SUM(totals.visits) AS Visits,
        SUM(totals.pageviews) AS PVs,
        SUM(totals.newVisits) AS NewVisits
    FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170102`
    GROUP BY
        date,
        device.deviceCategory
    )

SELECT *
FROM mult_ga_table
ORDER BY
    date ASC,
    Visits DESC
""")
# Way less efficient from a writing perspective (maybe more performant?)

In [10]:
ga_mult_day_df = pgbq.read_gbq(query_mult_days_union, dialect='standard')

In [11]:
ga_mult_day_df.head(3)

,date,deviceCategory,Visits,PVs,NewVisits
0,20170101,desktop,759,3246,576
1,20170101,mobile,503,1678,390
2,20170101,tablet,102,438,76


In [12]:
# Quick check that it returns similar data
table_suffix_sum = ga_mult_day_df[ga_mult_day_df['date'] == '20170101'].loc[:,"Visits"].sum()
union_table_sum = mult_day_df[mult_day_df['date'] == '20170101'].loc[:,"Visits"].sum()

table_suffix_sum == union_table_sum

True

In [13]:
# Hit sequence to hopefully get a feel for flattening
sequence_query = (
"""
SELECT
    fullVisitorId,
    visitId,
    visitNumber,
    hits.hitNumber AS hitNumber,
    hits.page.pagePath AS pagePath
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) as hits
WHERE
    _TABLE_SUFFIX BETWEEN '20170701' AND '20170702'
AND
    hits.type="PAGE"
ORDER BY
    fullVisitorId,
    visitId,
    visitNumber,
    hitNumber
""")

In [14]:
sequence_df = pgbq.read_gbq(sequence_query, dialect='standard')

In [15]:
sequence_df.head(10)

,fullVisitorId,visitId,visitNumber,hitNumber,pagePath
0,0000884932399915132,1499010813,1,1,/home
1,0000884932399915132,1499010813,1,2,/home
2,0000884932399915132,1499010813,1,3,/google+redesign/drinkware
3,0000884932399915132,1499010813,1,5,/google+redesign/drinkware/quickview
4,0000884932399915132,1499010813,1,6,/google+redesign/drinkware
5,0000884932399915132,1499010813,1,7,/google+redesign/drinkware
6,0000884932399915132,1499010813,1,8,/google+redesign/apparel
7,0000884932399915132,1499010813,1,9,/google+redesign/accessories
8,0001563469934876372,1498960250,1,1,/home
9,0001563469934876372,1498960250,1,2,/google+redesign/apparel/women+s+t+shirts/yout...


In [16]:
# Working on getting event data to define conversion as a metric (sessions where hit of xyz event >= 1)
event_query = (
"""
SELECT
    #visitId,
    trafficSource.campaign,
    device.deviceCategory,
    hits.eventInfo.eventCategory as EventCategory,
    hits.eventInfo.eventAction as EventAction,
    hits.eventInfo.eventLabel as EventLabel,
    SUM(totals.visits) AS Visits,
    SUM(totals.pageviews) AS PVs,
    SUM(totals.newVisits) AS NewVisits
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170101`,
    UNNEST(hits) as hits
WHERE
    trafficSource.campaign != "(not set)"
    AND 
    hits.eventInfo.eventLabel != "None"
GROUP BY
    #visitId,
    trafficSource.campaign,
    device.deviceCategory,
    EventCategory,
    EventAction,
    EventLabel
""")

In [17]:
event_hit_df = pgbq.read_gbq(event_query, dialect='standard')
# Need to utilize unnest to access the array data type

In [18]:
# So apparently you just unnest the hits in the from statement?
event_hit_df.head()

,campaign,deviceCategory,EventCategory,EventAction,EventLabel,Visits,PVs,NewVisits
0,Data Share Promo,tablet,Enhanced Ecommerce,Quickview Click,Google Tube Power Bank,1,4,1.0
1,Data Share Promo,mobile,Enhanced Ecommerce,Quickview Click,Windup Android,1,24,1.0
2,Data Share Promo,mobile,Enhanced Ecommerce,Quickview Click,Recycled Mouse Pad,1,24,1.0
3,AW - Dynamic Search Ads Whole Site,mobile,Enhanced Ecommerce,Quickview Click,Google Water Resistant Bluetooth Speaker,1,6,1.0
4,AW - Dynamic Search Ads Whole Site,mobile,Enhanced Ecommerce,Quickview Click,YouTube Leatherette Notebook Combo,1,7,1.0
